In [ ]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Fri Sep  4 13:20:25 2020
This script obtains web content.

@author: rachelcalder
@author: Jason Kim (edits for scraping DOM nodes)
colors = ["FF6666", "9999FF", "FF9933", "9966CC", "66CC66"]
DONE ART = FF6666
FILM = 9999FF 
SPECIAL = FF9933
LITERATURE = 9966CC
MUSIC = 66CC66
"""
import requests
from bs4 import BeautifulSoup 
import pandas as pd 

# Rachel - change years to your preference
final_table = pd.DataFrame(columns=["Name", "Date", "Time", "Price","Description", "Links"]) 
#print(final_table)
for etype in range(1, 6):
  for year in range(2004,2022):
    for month in range(1, 13):
      url = f"http://www.pd.org/~eyedrum/calendar/index.php?eventTypeId={etype}&month={month}&year={year}"
      #obtain website content
      response = requests.get(url)
      content = response.content
      #restructure url data for readability
      soup = BeautifulSoup(content, "lxml")

      # Jason's edits start here
      #print(f"Events for month={month} and year={year}")
      #print("=" * 50)
      # All events for this (year, month) are in a table with name=layoutOne, left column of the site
      leftColumn = soup.find("table", attrs={"name": "layoutOne"}).find("td")
      # Not having ID-ed nodes makes this hard, but all the events have the same CSS attribs
      events = leftColumn.find_all("table", attrs={"width": "300", "cellspacing": "0", "cellpadding": "0", "border": "0"})

      # We want the header w/ title, date, time, price, etc. Grab it with CSS background-color
      for event in events:
        # header = event.find("td", attrs={"bgcolor": "FF6666"})
        # Left of the header is a span w/ event name, right of the header is a span w/ date/time/price
        # Jason edit - don't need the additional .find, the spans we're looking for are the only ones
        # in each event anyway. Should work for all types of events now!
        spans = event.find_all("span")
        #print(spans)
        # Thus spans[0] is always the event name
        # Rachel - customize to however the data should be organized
        #print(f"Event Name: {spans[0].text}")
        e_name = spans[0].text
        if (len(spans) > 2):
          e_descr = spans[2].text
          #test_link = spans[2].find_all(href=True)
          test_link = []
          for link in spans[2].findAll('a'):
            test_link.append(link.get('href'))
        else:
          e_descr = "blank"
          test_link = "blank"
        # spans[1] containing date/time/price is trickier - convert <br> to \n to make splitting easier
        for br in spans[1].find_all("br"):
          br.replace_with("\n")
        # Last line is an empty string; remove. Replace non-breaking space w/ breaking space, too
        infoList = spans[1].text.strip().replace("\xa0", " ").split("\n")
        
        # infoList presumably has 1-3 elements, in the order of date, time, price
        # Rachel - I'm not sure what format you need in the end, so I left these as strings
        e_date = infoList[0]
        if (len(infoList) > 1):
          #print(f"Event Time: {infoList[1]}")
          e_time = infoList[1]
        else:
          e_time = "blank"
        if (len(infoList) > 2):
        # print(f"Event Pricing: {infoList[2]}")
          e_price = infoList[2]
        else:
          e_price = "blank"
        to_append = [e_name, e_date, e_time, e_price, e_descr, test_link]
        a_series = pd.Series(to_append, index = final_table.columns)
        final_table = final_table.append(a_series, ignore_index=True)
        
        #print()
      #print()

  print(final_table)
########################## REPLACE NAME OF THE FILE ##########
  final_table.to_csv("all_events_type"+str(etype)+"_0930.csv", escapechar="\r")
###########################################################





                     Name  ...                     Links
0              type = art  ...                        []
1                 earlier  ...                     blank
2     events sort by time  ...                     blank
3                   later  ...                     blank
4            HTML example  ...  [http://www.eyedrum.org]
...                   ...  ...                       ...
1617                 Hold  ...                     blank
1618                 Hold  ...                     blank
1619                 Hold  ...                     blank
1620                 Hold  ...                     blank
1621                 Hold  ...                     blank

[1622 rows x 6 columns]
                     Name  ...                     Links
0              type = art  ...                        []
1                 earlier  ...                     blank
2     events sort by time  ...                     blank
3                   later  ...                     blank
4     

In [ ]:
final_table['Description'][4]

"Open up an edit window on this event to see some basic HTML.\r A hyperlink:\rClick here to go to Eyedrum's website.\r Inserting an image:\r  Making a table (borders shown):\r A1A2A3 B1B2B3   Making a table (borders hidden):\r northwestnortheast southwestsoutheast   "